In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan 28 15:51:32 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              50W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
%%capture
!pip install transformers
!pip install datasets torchaudio librosa huggingface_hub jiwer
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install pyctcdecode
!pip install bitsandbytes
!pip install evaluate
!pip install wandb
!pip install jiwer
!pip install accelerate

In [3]:
%%capture
!pip install accelerate -U

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset, DatasetDict
import numpy as np
import pandas as pd
import os
from glob import glob

In [24]:
!git clone https://github.com/csikasote/bigcgen.git

Cloning into 'bigcgen'...
remote: Enumerating objects: 35681, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 35681 (delta 1), reused 8 (delta 1), pack-reused 35671 (from 4)
Receiving objects: 100% (35681/35681), 7.40 GiB | 18.05 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Updating files: 100% (35640/35640), done.


In [25]:
audio_path = os.path.join(os.getcwd(),"bigcgen/audio/")
csv_path = os.path.join(os.getcwd(),"bigcgen/splits/")

In [26]:
def remove_processed_files(csv_path):
  processed_files = glob(f"{csv_path}*/*_processed.tsv")
  len(processed_files)
  for f in processed_files:
    os.remove(f)
  print("Processed files removed successfully")

In [27]:
remove_processed_files(csv_path)

Processed files removed successfully


In [28]:
def prepare_data(audio_path, csv_path):
    split_list = ["male","female","balanced", "combined", "test"]
    for split in split_list:
        csv_file_list = glob(f"{csv_path}{split}/*.tsv")
        for csv_file in csv_file_list:
            split_file = os.path.basename(csv_file).split(".")[0]
            df = pd.read_csv(csv_file, sep="\t")
            df["path"] = audio_path + df['audio']
            df = df.dropna(subset=["path"])
            df = df.drop(columns=['audio'])
            df = df.rename(columns={'path':'audio'})
            df = df[["audio","sentence"]]
            df.to_csv(f"{csv_path}/{split}/{split_file}_processed.tsv", sep="\t", index=False)
            print(f"{split_file}_processed : ", len(df))

In [29]:
import numpy as np
from scipy.stats import f_oneway
def compute_OneWayANOVA():
  male_df = pd.read_csv(f"male_wer.csv")
  female_df = pd.read_csv(f"female_wer.csv")
  male_list = male_df.values.tolist()
  female_list = female_df.values.tolist()
  anova_results = f_oneway(male_list, female_list)
  sig_value = ''
  if anova_results[1][0] < 0.05:
    sig_value = 'True'
  else:
    sig_value = 'False'
  print("One-Way ANOVA:")
  print("F Statistic:",anova_results[0])
  print("P value:",anova_results[1], ":Significant:",sig_value)

In [30]:
prepare_data(audio_path, csv_path)

train_male_20hrs_file_processed :  10284
train_male_10hrs_file_processed :  5171
validation_male_file_processed :  441
test_native_and_nonnative_male_file_processed :  461
train_male_5hrs_file_processed :  2577
train_male_15hrs_file_processed :  7749
train_male_30hrs_file_processed :  15463
train_male_25hrs_file_processed :  12857
train_female_30hrs_file_processed :  16036
train_female_15hrs_file_processed :  8008
train_female_25hrs_file_processed :  13347
train_female_10hrs_file_processed :  5352
validation_female_file_processed :  475
train_female_20hrs_file_processed :  10674
train_female_5hrs_file_processed :  2675
test_native_and_nonnative_female_file_processed :  472
validation_balanced_file_processed :  442
train_balanced_file_processed :  2631
train_combined_10hrs_file_processed :  5252
train_combined_5hrs_file_processed :  2631
train_combined_40hrs_file_processed :  20958
train_combined_25hrs_file_processed :  13129
train_combined_30hrs_file_processed :  15757
train_combined_5

In [12]:
# Download the CODE from
!wget https://raw.githubusercontent.com/csikasote/ft_scripts/refs/heads/main/run_eval_whisper_modelv2.py
!wget https://raw.githubusercontent.com/csikasote/scripts/refs/heads/main/run_eval_whisper.sh
!wget https://raw.githubusercontent.com/csikasote/scripts/refs/heads/main/compute_ANOVA.py

--2025-01-28 15:54:06--  https://raw.githubusercontent.com/csikasote/ft_scripts/refs/heads/main/run_eval_whisper_modelv2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5886 (5.7K) [text/plain]
Saving to: ‘run_eval_whisper_modelv2.py’

run_eval_whisper_mo 100%[===================>]   5.75K  --.-KB/s    in 0s      

2025-01-28 15:54:07 (76.0 MB/s) - ‘run_eval_whisper_modelv2.py’ saved [5886/5886]

--2025-01-28 15:54:09--  https://raw.githubusercontent.com/csikasote/scripts/refs/heads/main/run_eval_whisper.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting

In [ ]:
model_id="csikasote/whisper-medium-bemgen-combined-model"
dataset="bemgen"
split="female"
file_name="combined"
csv_test_path = f"/content/{dataset}/splits/{split}/test_{file_name}_file_processed.tsv"
output_file = f"/content/{split}_output_wer.csv"

In [ ]:
!python run_eval_whisper_modelv2.py \
  --model_id=$model_id \
  --dataset=$dataset \
  --config="en" \
  --streaming="False" \
  --path=$csv_test_path \
  --output=$output_file \
  --device=0 \
  --language="en" \
  --batch_size=8 \
  --task="transcribe"

2025-01-27 09:26:51.302604: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 09:26:51.319295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-27 09:26:51.341082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-27 09:26:51.347459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 09:26:51.363026: I tensorflow/core/platform/cpu_feature_guar

In [31]:
# Run the model
!bash run_eval_whisper_single_model.sh

male
2025-01-28 17:45:33.720761: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-28 17:45:33.739083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-28 17:45:33.760264: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-28 17:45:33.766847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 17:45:33.782477: I tensorflow/core/platform/cpu_feature